### Connecting to Databricks-Azure

In [ ]:
import mlflow
import os 

# Set the Databricks workspace URL
mlflow.set_tracking_uri("databricks")



# Configure the connection to Databricks
os.environ["DATABRICKS_HOST"] = ""  # Replace with your Databricks URL
os.environ["DATABRICKS_TOKEN"] = ""  # Replace with your Databricks access token

In [ ]:
mlflow.set_experiment("")  # Adjust the path as needed

### Loading Dataset

In [ ]:
import pandas as pd

run_id = ""

artifact_uri = f"runs:/{run_id}/Telco-Customer-Churn-Clean.csv"

# Load the dataset using Pandas
df = pd.read_csv(mlflow.artifacts.download_artifacts(artifact_uri))

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

### Split data

In [ ]:
X = df.drop(columns=['Churn'])  
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Class imbalance

In [ ]:
from imblearn.combine import SMOTETomek
import numpy as np

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

### Hyper-parameter Tuning

In [ ]:
models = [
    (
        "Logistic Regression liblinear", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression()
    ),
    (
        "Logistic Regression lbfgs", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression()
    ),
    (
        "Random Forest est 30 depth 3", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier()
    ),
    (
        "Random Forest est 20 depth 4", 
        {"n_estimators": 20, "max_depth": 4},
        RandomForestClassifier()
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier()
    )
]

In [ ]:
reports = []

for model_name, params, model in models:
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [ ]:
reports

In [ ]:
for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        mlflow.set_tag("data_source", "Churn cleanded balanced")
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  